In [8]:
import numpy as np
import pandas as pd
import cv2
from glob import glob
import os
from natsort import natsorted
import matplotlib.pyplot as plt
import fly_analysis as fa
from scipy.signal import savgol_filter
from tqdm.contrib.concurrent import process_map
from pathlib import Path
import tqdm
import re
from scipy.stats import circmean
from scipy.interpolate import interp1d
pattern = r'.*?(\d+)_obj_id_(\d+)_cam_(\d+)_frame_(\d+)\.[^.]+$'

import faulthandler
faulthandler.enable()

In [15]:
def normalize_angle(angle):
    """Normalize angle to [-π, π]"""
    return (angle + np.pi) % (2 * np.pi) - np.pi

def interpolate_heading_circular(df, stim_value):
    # Find the rows where stim_value falls between
    lower_row = df[df['stim_direction'] <= stim_value].iloc[-1]
    upper_row = df[df['stim_direction'] >= stim_value].iloc[0]
    
    # Get the angles
    angle1 = lower_row['heading_direction']
    angle2 = upper_row['heading_direction']
    
    # Calculate the direct angular difference
    diff = angle2 - angle1
    
    # If the difference is greater than π, we need to go the other way around
    if abs(diff) > np.pi:
        if diff > 0:
            angle2 -= 2 * np.pi
        else:
            angle2 += 2 * np.pi
    
    # Calculate interpolation factor
    t = (stim_value - lower_row['stim_direction']) / (upper_row['stim_direction'] - lower_row['stim_direction'])
    
    # Interpolate heading
    heading = angle1 + t * (angle2 - angle1)
    
    # Normalize the result back to [-π, π]
    return normalize_angle(heading)

def calculate_velocities(df, dt=0.002):  # dt = 1/500 for 500Hz
    # Calculate velocities using gradient with explicit time step
    df['xvel'] = np.gradient(df['x'], dt)  # pixels per second
    df['yvel'] = np.gradient(df['y'], dt)  # pixels per second
    
    # Calculate heading (velocity direction in degrees)
    df['heading'] = np.degrees(np.arctan2(df['yvel'], df['xvel']))
        
    return df

def process_video(video_path):
    # Open the video file
    cap = cv2.VideoCapture(str(video_path))
    
    # Get total number of frames
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Initialize numpy array with NaN values
    results = np.full((total_frames, 4), np.nan)
    # Fill the first column with frame numbers
    results[:, 0] = np.arange(total_frames)
    
    frame_number = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Apply threshold
        _, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY_INV)
        
        # Find contours
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if contours:
            # Get the largest contour
            largest_contour = max(contours, key=cv2.contourArea)
            
            # Try to fit an ellipse
            if len(largest_contour) >= 5:  # Need at least 5 points to fit an ellipse
                ellipse = cv2.fitEllipse(largest_contour)
                
                # Extract information
                (x, y), (MA, ma), angle = ellipse
                
                # Update the corresponding row in results
                results[frame_number, 1:] = [x, y, angle]
        
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        frame_number += 1
    
    # Release the video capture object
    cap.release()
    
    # Convert results to a pandas DataFrame with headers
    df = pd.DataFrame(results, columns=['frame', 'x', 'y', 'orientation'])
    
    # Calculate velocities and heading
    df = calculate_velocities(df)

    return df


def sg_smooth(arr):
    return savgol_filter(arr, 51, 3)

In [10]:
root_folder = "/home/buchsbaum/mnt/nfc3008/md0/"
exp_file = "20230626_161309"
videos_folder = "/home/buchsbaum/Videos/20230626_161309/"
braidz_file = root_folder + f"/Experiments/{exp_file}.braidz"
calibration_videos_folder = root_folder + "/Videos/calibration/20230626_pixel_to_direction_calibration_"
stim2camera = pd.read_csv("stim2camera.csv")

In [11]:
video_files = natsorted(glob(os.path.join(videos_folder, "*.mp4")))
calibration_video_files = natsorted(glob(os.path.join(calibration_videos_folder, "*.mp4")))

In [12]:
data = fa.braidz.read_braidz(braidz_file)

Reading /home/buchsbaum/mnt/nfc3008/md0//Experiments/20230626_161309.braidz using pyarrow


In [13]:
stim = data[1]["stim"]

In [16]:
# Process video files
for video_path in video_files:
    df = process_video(video_path)

    # Save CSV file
    csv_filename = video_path.replace('.mp4', '.csv')
    df.to_csv(csv_filename, index=False, float_format='%.6f')
    break

: 

In [ ]:
csv_files = natsorted(glob(os.path.join(videos_folder, "*.csv")))

['/gpfs/soma_fs/nfc/nfc3008//Videos/20230626_161309/1_obj_id_33646_cam_23047980_frame_1129843.csv', '/gpfs/soma_fs/nfc/nfc3008//Videos/20230626_161309/1_obj_id_33646_cam_23088879_frame_1129843.csv', '/gpfs/soma_fs/nfc/nfc3008//Videos/20230626_161309/1_obj_id_33646_cam_23088882_frame_1129843.csv', '/gpfs/soma_fs/nfc/nfc3008//Videos/20230626_161309/1_obj_id_33646_cam_23096298_frame_1129843.csv', '/gpfs/soma_fs/nfc/nfc3008//Videos/20230626_161309/2_obj_id_35921_cam_23047980_frame_1206880.csv', '/gpfs/soma_fs/nfc/nfc3008//Videos/20230626_161309/2_obj_id_35921_cam_23088879_frame_1206880.csv', '/gpfs/soma_fs/nfc/nfc3008//Videos/20230626_161309/2_obj_id_35921_cam_23088882_frame_1206880.csv', '/gpfs/soma_fs/nfc/nfc3008//Videos/20230626_161309/2_obj_id_35921_cam_23096298_frame_1206880.csv', '/gpfs/soma_fs/nfc/nfc3008//Videos/20230626_161309/3_obj_id_36053_cam_23047980_frame_1227209.csv', '/gpfs/soma_fs/nfc/nfc3008//Videos/20230626_161309/3_obj_id_36053_cam_23088879_frame_1227209.csv', '/gpfs/so

In [ ]:
stim_diff = []
heading_diff = []
stim_duration = []
stim_radius = []

for csv_file in csv_files:
    
    # get video tracking data
    video_df = pd.read_csv(csv_file)

    # get object id, camera number, and frame number
    match = re.search(pattern, csv_file)
    prefix, obj_id, cam, frame = match.groups()
    stim_row = stim_data[(stim_data["obj_id"] == int(obj_id)) & (stim_data["frame"] == int(frame))]

    # get stimulus data
    stimulus_pos_x = stim_row["looming_pos_x"].values[0]
    stimulus_radius = stim_row["looming_radius"].values[0]
    stimulus_duration = stim_row["looming_duration"].values[0]

    # get the stimulus direction in angles
    camera_calibration_df = stim2camera[stim2camera["camera"] == int(cam)]

    # interpolate the heading direction
    stimulus_pos_heading = interpolate_heading_circular(camera_calibration_df, stimulus_pos_x)
    
    stim_direction_in_pixel = camera_calibration_df["stim_direction"].values
    stim_direction_in_heading = camera_calibration_df["heading_direction"].values

    interp_stim_direction = np.interp(stimulus_pos_x, stim_direction_in_pixel, stim_direction_in_heading)
    
    # get heading of fly before stimulus
    x = sg_smooth(video_df["x"].values)
    y = sg_smooth(video_df["y"].values)
    xvel = sg_smooth(video_df["xvel"].values)
    yvel = sg_smooth(video_df["yvel"].values)

    theta = np.arctan2(yvel, xvel)

    theta_before = theta[450:600]
    theta_before = theta_before[~np.isnan(theta_before)]
    
    theta_after = theta[650:700]
    theta_after = theta_after[~np.isnan(theta_after)]

    theta_before_mean = circmean(theta_before, high=np.pi, low=-np.pi)
    theta_after_mean = circmean(theta_after, high=np.pi, low=-np.pi)

    fly_stimulus_diff = fa.trajectory.angdiff(theta_after_mean, stimulus_pos_heading, 'rad')
    fly_fly_diff = fa.trajectory.angdiff(theta_after_mean, theta_before_mean, 'rad')

    stim_diff.append(fly_stimulus_diff)
    heading_diff.append(fly_fly_diff)
    stim_duration.append(stimulus_duration)
    stim_radius.append(stimulus_radius)


out_df = {
    "stim_diff": stim_diff,
    "heading_diff": heading_diff,
    "stim_duration": stim_duration,
    "stim_radius": stim_radius
}

In [174]:
camera_calibration_df

,camera,stim_direction,heading_direction
0,23047980,0,-2.484245
1,23047980,80,2.984215
2,23047980,160,2.689015
3,23047980,240,1.936260
4,23047980,320,1.314023
5,23047980,400,0.258200
6,23047980,480,-0.189250
7,23047980,560,-1.154501
